In [183]:
import pandas as pd
import glob
import os
from datetime import datetime, timedelta

In [184]:
months = ["03", "04", "05", "06", "07", "08", "09"]

In [185]:
def clean_delay_to_minutes(sec):
    if sec < 0:
        return 0.0
    else:
        return sec/60.0

In [186]:
def adjust_actual_times(row):
    row["delay_minutes"] = clean_delay_to_minutes(row["delay_minutes"])

    if row["delay_minutes"] > 700:
        
        # if delay ~= 1400, parsing error, subtract full day
        if row["delay_minutes"] > 1400:
            row["actual_time"] = row["actual_time"] - timedelta(hours=24)
        
        # if delay ~= 720, parsing error, subtract half day
        else:
            row["actual_time"] = row["actual_time"] - timedelta(hours=12)
        
        # recompute delay
        row["delay_minutes"] = (row["actual_time"] - row["scheduled_time"]).total_seconds()
        row["delay_minutes"] = clean_delay_to_minutes(row["delay_minutes"])

        return row
    
    return None

In [187]:
for month in months:
    
    # read in CSVs for month
    path = "/Users/pranav/Documents/development/njtransit/csv/2018_{}".format(month)
    monthFiles = glob.glob(os.path.join(path, "*.csv"))
    df = pd.concat((pd.read_csv(f) for f in sorted(monthFiles)))
    print("done reading csv {}".format(month))
    
    # series-wide operations
    df.columns = ["train_id", "date", "stop_sequence", "from", "from_id", "to", "to_id", "scheduled_time", "actual_time", "status", "line", "type"]
    df['scheduled_time'] = pd.to_datetime(df['scheduled_time'])
    df['actual_time'] = pd.to_datetime(df['actual_time'])
    df['delay_minutes']=(df['actual_time']-df['scheduled_time']).dt.total_seconds()
    print("done series-wide {}".format(month))
    
    # adjust/clean times
    df_records = df.to_dict('records')
    for idx, record in enumerate(df_records):
        record = adjust_actual_times(record)
        if record is not None:
            df_records[idx] = record
    print("done adjust/clean {}".format(month))
    
    # generate cleaned df
    df = pd.DataFrame(df_records)
    columns = ["date", "train_id", "stop_sequence", "from", "from_id", "to", "to_id", "scheduled_time", "actual_time", "delay_minutes", "status", "line", "type"]
    df = df[columns]
    df.to_csv("/Users/pranav/Documents/development/njtransit/csv/monthly/2018_{}.csv".format(month), index=False)
    print("done generating df {}".format(month))

done reading csv 03
done series-wide 03
done adjust/clean 03
done generating df 03
done reading csv 04
done series-wide 04
done adjust/clean 04
done generating df 04
done reading csv 05
done series-wide 05
done adjust/clean 05
done generating df 05
done reading csv 06
done series-wide 06
done adjust/clean 06
done generating df 06
done reading csv 07
done series-wide 07
done adjust/clean 07
done generating df 07
done reading csv 08
done series-wide 08
done adjust/clean 08
done generating df 08
done reading csv 09
done series-wide 09
done adjust/clean 09
done generating df 09


In [188]:
df.head()

,date,train_id,stop_sequence,from,from_id,to,to_id,scheduled_time,actual_time,delay_minutes,status,line,type
0,2018-09-01,0720,1.0,Gladstone,49.0,Gladstone,49.0,2018-09-01 10:52:00,2018-09-01 10:52:06,0.100000,departed,Gladstone Branch,NJ Transit
1,2018-09-01,0720,2.0,Gladstone,49.0,Peapack,117.0,2018-09-01 10:55:00,2018-09-01 10:56:10,1.166667,departed,Gladstone Branch,NJ Transit
2,2018-09-01,0720,3.0,Peapack,117.0,Far Hills,45.0,2018-09-01 10:59:00,2018-09-01 11:01:31,2.516667,departed,Gladstone Branch,NJ Transit
3,2018-09-01,0720,4.0,Far Hills,45.0,Bernardsville,18.0,2018-09-01 11:05:00,2018-09-01 11:07:02,2.033333,departed,Gladstone Branch,NJ Transit
4,2018-09-01,0720,5.0,Bernardsville,18.0,Basking Ridge,12.0,2018-09-01 11:08:00,2018-09-01 11:10:15,2.250000,departed,Gladstone Branch,NJ Transit
